In [1]:
from requests import get
from bs4 import BeautifulSoup
import re
import pandas as pd
from time import sleep
from random import randint
from time import time
from IPython.core.display import clear_output
from warnings import warn
import pandas as pd
import numpy as np
import math
import os
import unicodedata
import string
import time
from functools import wraps
from colorama import Fore
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotVisibleException
from requests import ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError
import datetime
from dateutil.parser import parse

In [2]:
import pandas as pd
import numpy as np
import datetime
import dateutil
from datetime import timedelta
from IPython.core.display import clear_output
from collections import Counter
pd.set_option('display.max_columns', 500)
import ast
from collections import Counter
import re
from tqdm.notebook import tqdm, tnrange
tqdm.pandas()

from PIL import Image
import requests
import bs4

/Users/brianphelps/opt/anaconda3/envs/thesis/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
def retry(ExceptionToCheck, tries=20, delay=3, backoff=2, logger=None):
    """Retry calling the decorated function using an exponential backoff.

    http://www.saltycrane.com/blog/2009/11/trying-out-retry-decorator-python/
    original from: http://wiki.python.org/moin/PythonDecoratorLibrary#Retry
    
    :param ExceptionToCheck: the exception to check. may be a tuple of
        exceptions to check
    :type ExceptionToCheck: Exception or tuple
    :param tries: number of times to try (not retry) before giving up
    :type tries: int
    :param delay: initial delay between retries in seconds
    :type delay: int
    :param backoff: backoff multiplier e.g. value of 2 will double the delay
        each retry
    :type backoff: int
    :param logger: logger to use. If None, print
    :type logger: logging.Logger instance
    """
    def deco_retry(f):

        @wraps(f)
        def f_retry(*args, **kwargs):
            mtries, mdelay = tries, delay
            while mtries > 1:
                try:
                    return f(*args, **kwargs)
                except ExceptionToCheck:
                    msg = "%s, Retrying in %d seconds..." % (str(ExceptionToCheck), mdelay)
                    if logger:
                        #logger.exception(msg) # would print stack trace
                        logger.warning(msg)
                    else:
                        print(msg)
                    time.sleep(mdelay)
                    mtries -= 1
                    mdelay *= backoff
            return f(*args, **kwargs)

        return f_retry  # true decorator

    return deco_retry  

In [11]:
options = webdriver.ChromeOptions()
options.add_argument("no-sandbox")
options.add_argument("disable-dev-shm-usage")
options.add_argument("headless")
options.add_argument("user-data-dir=/Users/brianphelps/Library/Application Support/Google/Chrome/Thread_Swoops")

In [22]:
# Store data in lists
def scrapeMovies(data, saved_data=False):
  #for every page
  count=0
  if type(saved_data)!=bool:
    add_feats = saved_data
    data = data.iloc[len(add_feats):]
  else:
    add_feats = pd.DataFrame()
  for row in data.itertuples():
    name = row.name
    year = row.year
    profile = row.profile
    count+=1

    link = f"https://www.imdb.com{profile}"
    
    @retry((ConnectionResetError, ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError), tries=20, delay=2,backoff=2)
    def get_response():
      response = get(link)
      return response
    response = get_response()

    # parse the content of request
    page_html = BeautifulSoup(response.text, 'html.parser')

    print(Fore.GREEN + f"{name}({count})\n")

    '''
      GRAB PLOT SYNOPSIS
    '''
    synopsis_link = profile
    @retry((ConnectionResetError, ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError), tries=20, delay=2, backoff=2)
    def get_synopsis():
      synopsis = get("https://www.imdb.com/" + synopsis_link + "plotsummary")
      return synopsis
    synopsis = get_synopsis()
    synopsis_html = BeautifulSoup(synopsis.text, 'html.parser')
    
    try:
      plot_synopsis_content = synopsis_html.find("ul", {"id": "plot-synopsis-content"}).li.text
      plot_synopsis_content = plot_synopsis_content.strip()
      if plot_synopsis_content[:58] != "It looks like we don't have a Synopsis for this title yet.":
        print(Fore.GREEN + 'Plot Synopsis\n')
      else:
        plot_synopsis_content = np.nan
        print(Fore.RED + 'Plot Synopsis\n')
    except:
      plot_synopsis_content = np.nan
      print(Fore.RED + 'Plot Synopsis\n')
    '''
        GRAB SUMMARIES
    '''

    try:
      summary_bucket = []
      summaries = synopsis_html.find('ul', attrs={"id":"plot-summaries-content"}).find_all('li')
      for summary in summaries:
        result = summary.p.text.strip()
        summary_bucket.append(result)
      summaries = summary_bucket
      print(Fore.GREEN + f'Summaries: {len(summary_bucket)}\n')
    except:
      summaries = np.nan
      print(Fore.RED + 'Summary\n')

   
    '''
      MOVIE POSTER
    '''
    driver = webdriver.Chrome("/Users/brianphelps/Desktop/chromedriver", chrome_options=options)
    try:
      image_url = f"https://www.imdb.com/{page_html.find('div', class_='poster').a['href']}"
      @retry((ConnectionResetError, ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError), tries=20, delay=2,backoff=2)
      def get_response(link):
        driver.get(link)
      get_response(image_url)
      poster_html = BeautifulSoup(driver.page_source, 'html.parser')
      
      try:
        image_url = poster_html.find_all('div', class_='pswp__item')[1].find('img', class_='pswp__img')['src']
      except:
        image_url = poster_html.find_all('img')[1]['src']
      
      movie_name = name.replace(" ", "_")
      
      img = Image.open(requests.get(image_url, stream = True).raw)
      try:
        img.save(f'./posters/{movie_name}_{year}.jpg')
      except:
        img.convert('RGB').save(f'./posters/{movie_name}_{year}.jpg')
        
      poster = f"./posters/{movie_name}_{year}.jpg"
      print(Fore.GREEN + 'Poster\n')
    except Exception as e:
      poster = np.nan
      print(e)
      print(Fore.RED + 'Poster\n')
      
    driver.quit()
    movie = pd.DataFrame({
      'name': [name],
      'profile': [profile],
      'synopsis': [plot_synopsis_content],
      'summaries': [summaries],
      'poster': [poster]
    })
    add_feats = add_feats.append(movie, ignore_index=True)
    add_feats.to_csv(f"./add_feats.csv", index=False)
    print(f"Movies:  {len(add_feats)}\n")
    clear_output(wait = True)
  return add_feats

In [23]:
df = pd.read_csv("./data.csv")

In [ ]:
movies = scrapeMovies(df)

Missionary(10653)

Plot Synopsis

Summaries: 1

